In [55]:
import string
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from nltk.tokenize import StanfordTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## read in the character list
master = pd.read_csv('Shakespeare_characters.txt', delimiter='\t', encoding='ISO-8859-1')
master.head()

,Speaker,Unnamed: 1,Gender,#Lines,Play,TopVillain,Fools
0,Hamlet,NaN,Male,1506,Hamlet,NaN,NaN
1,Iago,NaN,Male,1088,Othello,yes,NaN
2,King Henry,NaN,Male,1031,Henry V,NaN,NaN
3,Othello,NaN,Male,880,Othello,NaN,NaN
4,Timon,NaN,Male,850,Timon of Athens,NaN,NaN


In [3]:
master.tail()

,Speaker,Unnamed: 1,Gender,#Lines,Play,TopVillain,Fools
1528,First And Second Murderer,NaN,Male,1,Richard III,NaN,NaN
1529,All,NaN,Male,1,Richard III,NaN,NaN
1530,Ghost Of Grey,NaN,Male,1,Richard III,NaN,NaN
1531,Herbert,NaN,Male,1,Richard III,NaN,NaN
1532,Queen Elizabeth And Duchess Of York,NaN,Female,1,Richard III,NaN,NaN


In [4]:
home_url = 'http://shakespeare.mit.edu/index.html'
request = requests.get(home_url).content
main_soup = BeautifulSoup(request, 'html.parser')

In [5]:
play_hrefs = [a['href'] for a in main_soup.findAll('a', href=True) if 'index.html' in a['href']]

In [6]:
main_url = 'http://shakespeare.mit.edu/'

In [17]:
## still need to get rid of the stage direction
def get_char_text(href, play):
    
    new_url = 'http://shakespeare.mit.edu/' + href
    request = requests.get(new_url).content
    new_soup = BeautifulSoup(request, 'html.parser')
    
    act, scene = re.compile('\.(\d+)\.(\d+)\.html').findall(href)[0]

    characters = [b.text for b in new_soup.findAll('b')]
    quotes = [quote.text.split('\n') for quote in new_soup.findAll('blockquote')]
    quotes = quotes[1:]
    
    character_text = [i for i in zip(characters, quotes)]    
    
    dict_list = []
    for i in character_text:
        character = i[0]
        the_text = i[1]
        for j in the_text:
            dict_list.append({'play': play, 'act': act, 'scene': scene, 
                              'character': character, 'text': j})
    return(dict_list)

In [21]:
all_dicts = []
for play_href in play_hrefs:
    play = re.compile('(\w+)\/index.html').findall(play_href)[0]
    print(play)
    print(main_url + play_href)
    request = requests.get(main_url + play_href).content
    the_soup = BeautifulSoup(request, 'html.parser')
    inside_hrefs = [play + '/' + a['href'] for a in the_soup.findAll('a', href=True) if play in a['href']]
    temp_dicts = []
    for href in inside_hrefs:
        dict_list = get_char_text(href, play)
        for i in dict_list:
            temp_dicts.append(i)
    for the_dict in temp_dicts:
        all_dicts.append(the_dict)

allswell
http://shakespeare.mit.edu/allswell/index.html
asyoulikeit
http://shakespeare.mit.edu/asyoulikeit/index.html
comedy_errors
http://shakespeare.mit.edu/comedy_errors/index.html
cymbeline
http://shakespeare.mit.edu/cymbeline/index.html
lll
http://shakespeare.mit.edu/lll/index.html
measure
http://shakespeare.mit.edu/measure/index.html
merry_wives
http://shakespeare.mit.edu/merry_wives/index.html
merchant
http://shakespeare.mit.edu/merchant/index.html
midsummer
http://shakespeare.mit.edu/midsummer/index.html
much_ado
http://shakespeare.mit.edu/much_ado/index.html
pericles
http://shakespeare.mit.edu/pericles/index.html
taming_shrew
http://shakespeare.mit.edu/taming_shrew/index.html
tempest
http://shakespeare.mit.edu/tempest/index.html
troilus_cressida
http://shakespeare.mit.edu/troilus_cressida/index.html
twelfth_night
http://shakespeare.mit.edu/twelfth_night/index.html
two_gentlemen
http://shakespeare.mit.edu/two_gentlemen/index.html
winters_tale
http://shakespeare.mit.edu/winters_

In [22]:
text_df = pd.DataFrame(all_dicts)
blank_filter = text_df["text"] != ""
text_df = text_df[blank_filter]
text_df = text_df[['play', 'act','scene', 'character', 'text']]
text_df = text_df.sort_values(by=['act', 'scene'])
text_df.head()

,play,act,scene,character,text
81578,2henryiv,0,0,RUMOUR,"Enter RUMOUR, painted full of tongues"
49051,taming_shrew,0,1,SLY,"I'll pheeze you, in faith."
49054,taming_shrew,0,1,Hostess,"A pair of stocks, you rogue!"
49057,taming_shrew,0,1,SLY,Ye are a baggage: the Slys are no rogues; look in
49058,taming_shrew,0,1,SLY,the chronicles; we came in with Richard Conque...


In [23]:
text_df.tail()

,play,act,scene,character,text
63197,troilus_cressida,5,9,AGAMEMNON,March patiently along: let one be sent
63198,troilus_cressida,5,9,AGAMEMNON,To pray Achilles see us at our tent.
63199,troilus_cressida,5,9,AGAMEMNON,"If in his death the gods have us befriended,"
63200,troilus_cressida,5,9,AGAMEMNON,"Great Troy is ours, and our sharp wars are ended."
63201,troilus_cressida,5,9,AGAMEMNON,"Exeunt, marching"


In [24]:
text_df.shape

(113332, 5)

In [25]:
text_df['character'] = text_df.character.apply(lambda x: str.lower(x))
text_df['text'] = text_df.text.apply(lambda x: str.lower(x))

In [26]:
text_df.head()

,play,act,scene,character,text
81578,2henryiv,0,0,rumour,"enter rumour, painted full of tongues"
49051,taming_shrew,0,1,sly,"i'll pheeze you, in faith."
49054,taming_shrew,0,1,hostess,"a pair of stocks, you rogue!"
49057,taming_shrew,0,1,sly,ye are a baggage: the slys are no rogues; look in
49058,taming_shrew,0,1,sly,the chronicles; we came in with richard conque...


In [30]:
pd.to_pickle(text_df, 'text_df.pkl')

In [31]:
all_characters = list(set(text_df.character.values))

In [66]:
def get_all_text(character, scraped_df):
    character = scraped_df[scraped_df.character == character]
    character_text = character.text.values
    exclude_text = set(string.punctuation)
    character_text = [''.join(ch for ch in i if ch not in exclude) for i in character_text] ## removes punctuation
    wnl = WordNetLemmatizer()
    character_text = [wnl.lemmatize(j) for i in character_text for j in i.split()]
    character_text = [word for word in character_text if word not in stopwords.words('english')]
    return(character_text)

In [69]:
prepped_dict = {}
for character in all_characters:
    prepped_dict[character] = get_all_text(character, text_df)